In [1]:
import pandas as pd 
import numpy as np 
import jieba # pip install jieba

from tensorflow.keras.layers import Dense, Input, Flatten, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding

In [2]:
# 读入数据
neg=pd.read_excel('data/neg.xls',header=None)
pos=pd.read_excel('data/pos.xls',header=None)

In [3]:
#合并语料
pn = pd.concat([pos,neg],ignore_index=True) 
#计算语料数目
neglen = len(neg)
print(neglen)
poslen = len(pos) 
print(poslen)

10428
10677


In [4]:
#定义分词函数
cw = lambda x: list(jieba.cut(x))
pn['words'] = pn[0].apply(cw)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DELL\AppData\Local\Temp\jieba.cache
Loading model cost 0.603 seconds.
Prefix dict has been built successfully.


In [5]:
pn

,0,words
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...,"[做, 父母, 一定, 要, 有, 刘墉, 这样, 的, 心态, ，, 不断, 地, 学习,..."
1,作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...,"[作者, 真有, 英国人, 严谨, 的, 风格, ，, 提出, 观点, 、, 进行, 论述,..."
2,作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...,"[作者, 长篇大论, 借用, 详细, 报告, 数据处理, 工作, 和, 计算结果, 支持, ..."
3,作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...,"[作者, 在, 战, 几时, 之前, 用, 了, ＂, 拥抱, ＂, 令人, 叫绝, ．, ..."
4,作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...,"[作者, 在, 少年, 时即, 喜, 阅读, ，, 能, 看出, 他, 精读, 了, 无数,..."
...,...,...
21100,安装我自己花了500多，美的够黑心的，真的是烦心，安装的售后叼的要死！差评！！！！！,"[安装, 我, 自己, 花, 了, 500, 多, ，, 美的, 够, 黑心, 的, ，, ..."
21101,东西不错，售后太差，安装一个热水器400块钱，像话吗？钱给完了发现被黑了！,"[东西, 不错, ，, 售后, 太, 差, ，, 安装, 一个, 热水器, 400, 块钱,..."
21102,碰到最差的、最骗人的卖家，好吧，我倒霉！这个卖家太不厚道了，显示所在地上海，我买这个热水器选...,"[碰到, 最差, 的, 、, 最, 骗人, 的, 卖家, ，, 好, 吧, ，, 我, 倒霉..."
21103,宝贝不错，物流也不错，售后差，,"[宝贝, 不错, ，, 物流, 也, 不错, ，, 售后, 差, ，]"


In [6]:
# 一行数据最多的词汇数
max_document_length = max([len(x) for x in pn['words']])
max_document_length

1804

In [7]:
# 设置一个评论最多1000个词
max_document_length = 1000

In [8]:
texts = [' '.join(x) for x in pn['words']]
# 查看一条评论
texts[-2]

'宝贝 不错 ， 物流 也 不错 ， 售后 差 ，'

In [9]:
# 实例化分词器，设置字典中最大词汇数为30000
tokenizer = Tokenizer(num_words=30000)
# 传入我们的训练数据，建立词典
tokenizer.fit_on_texts(texts) 
# 把词转换为编号，词的编号根据词频设定，频率越大，编号越小
sequences = tokenizer.texts_to_sequences(texts) 
# 把序列设定为1000的长度，超过1000的部分舍弃，不到1000则补0
sequences = pad_sequences(sequences, maxlen=1000)  
sequences = np.array(sequences)

In [10]:
# 词对应编号的字典
dict_text = tokenizer.word_index
dict_text['也']

9

In [11]:
sequences[-2]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [12]:
# 定义标签
positive_labels = [[0, 1] for _ in range(poslen)]
negative_labels = [[1, 0] for _ in range(neglen)]
y = np.concatenate([positive_labels, negative_labels], 0)

# 打乱数据
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = sequences[shuffle_indices]
y_shuffled = y[shuffle_indices]

# 数据集切分为两部分
test_sample_index = -1 * int(0.1 * float(len(y)))
x_train, x_test = x_shuffled[:test_sample_index], x_shuffled[test_sample_index:]
y_train, y_test = y_shuffled[:test_sample_index], y_shuffled[test_sample_index:]

In [15]:
from tensorflow.python.keras.layers.wrappers import Bidirectional
from tensorflow.python.keras.layers.recurrent import LSTM
from tensorflow.python.keras.layers.recurrent import GRU
# 定义函数式模型
# 模型输入
sequence_input = Input(shape=(1000,))
# Embedding层，30000表示30000个词，每个词对应的向量为128维，序列长度为1000
embedding_layer = Embedding(30000,
                            128,
                            input_length=1000)
embedded_sequences = embedding_layer(sequence_input)

# 方法一，LSTM的输出为(batch, 10)
lstm1 = LSTM(10, dropout=0.2, recurrent_dropout=0.2)(embedded_sequences)
lstm1 = Dense(16, activation='relu')(lstm1)
lstm1 = Dropout(0.5)(lstm1)

# 方法二，LSTM的输出为(batch, 1000, 10)
# lstm1 = LSTM(10,return_sequences=True,dropout=0.2, recurrent_dropout=0.2)(embedded_sequences)
# lstm1 = Flatten()(lstm1)
# lstm1 = Dense(16, activation='relu')(lstm1)
# lstm1 = Dropout(0.5)(lstm1)

# 方法三
# 可以结合采用多种RNN模型
# lstm2 = Bidirectional(LSTM(10,return_sequences=True,dropout=0.2, recurrent_dropout=0.2))(embedded_sequences)
# lstm2 = Flatten()(lstm2)
# lstm2 = Dense(16, activation='relu')(lstm2)
# lstm2 = Dropout(0.5)(lstm2)

# lstm3 = GRU(10,return_sequences=True,dropout=0.2, recurrent_dropout=0.2)(embedded_sequences)
# lstm3 = Flatten()(lstm3)
# lstm3 = Dense(16, activation='relu')(lstm3)
# lstm3 = Dropout(0.5)(lstm3)
# merge = concatenate([lstm1, lstm2, lstm3], axis=1)
# merge=Dense(16,activation='relu')(merge)
# merge=Dropout(0.5)(merge)

# 输出层
preds = Dense(2, activation='softmax')(lstm1)
# 定义模型
model = Model(sequence_input, preds)

In [16]:
# 训练模型
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_test, y_test))

Epoch 1/5
149/149 [==============================] - 285s 2s/step - loss: 0.5449 - acc: 0.7446 - val_loss: 0.2974 - val_acc: 0.8915
Epoch 2/5
149/149 [==============================] - 343s 2s/step - loss: 0.2876 - acc: 0.9007 - val_loss: 0.2514 - val_acc: 0.9100
Epoch 3/5
149/149 [==============================] - 337s 2s/step - loss: 0.1799 - acc: 0.9489 - val_loss: 0.2380 - val_acc: 0.9199
Epoch 4/5
149/149 [==============================] - 344s 2s/step - loss: 0.1243 - acc: 0.9702 - val_loss: 0.2587 - val_acc: 0.9204
Epoch 5/5
149/149 [==============================] - 347s 2s/step - loss: 0.0885 - acc: 0.9794 - val_loss: 0.2793 - val_acc: 0.9161


In [17]:
# 预测
def predict(text):
    # 对句子分词
    cw = list(jieba.cut(text)) 
    word_id = []
    # 把词转换为编号
    for word in cw:
        try:
            temp = dict_text[word]
            word_id.append(temp)
        except:
            word_id.append(0)
    word_id = np.array(word_id)
    word_id = word_id[np.newaxis,:]
    sequences = pad_sequences(word_id, maxlen=1000, padding='post')  
    result = np.argmax(model.predict(sequences))
    if(result==1):
        print("positive comment")
    else:
        print("negative comment")
    

In [18]:
predict("东西质量不错，下次还会再来买")

positive comment
